Name: Nikhil Rajendra Dhumal

Roll No.: 21102A0009

Class: BE CMPN A

### Imports

In [1]:
import nltk, re, pprint
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\treebank.zip.


True

## Training Dataset (Corpus)

In [2]:
# Custom train set
train_set = [[('Martin', 'NNP'), (',', ','), ('Justin', 'NNP'), ('can', 'MD'), ('watch', 'VBN'), ('will', 'NNP'), ('.', '.')], [('Spot', 'NNP'), ('will', 'MD'), ('watch', 'VBN'), ('Martin', 'NNP'), ('.', '.')], [('Will', 'MD'), ('Justin', 'NNP'), ('spot', 'VBN'), ('Martin', 'NNP'), ('.', '.')], [('Martin', 'NNP'), ('will', 'MD'), ('pat', 'VBN'), ('Spot', 'NNP'), ('.', '.')]]

In [3]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

22

In [4]:
# tokens
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Martin', ',', 'Justin', 'can', 'watch', 'will', '.', 'Spot', 'will', 'watch']

In [5]:
# vocabulary
V = set(tokens)
print(len(V))

# number of tags
T = set([pair[1] for pair in train_tagged_words])
print(T)

11
{'MD', ',', 'VBN', 'NNP', '.'}


In [6]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

## Emission Probability

In [7]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)

    return (count_w_given_tag, count_tag)

## Transition Probability

In [8]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

## Transition Probability Matrix

In [9]:
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,MD,",",VBN,NNP,.
MD,0.000000,0.000000,0.750000,0.25,0.000000
",",0.000000,0.000000,0.000000,1.00,0.000000
VBN,0.000000,0.000000,0.000000,1.00,0.000000
NNP,0.333333,0.111111,0.111111,0.00,0.444444
.,0.250000,0.000000,0.000000,0.50,0.000000


In [10]:
tags_df.loc['.', :]

MD     0.25
,      0.00
VBN    0.00
NNP    0.50
.      0.00
Name: ., dtype: float32

## POS Tagging on Test Data

In [11]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

In [12]:
corpus = "Jack will spot will"
test_tagged_words = corpus.split()

# Expected result
expected_result = [('Jack', 'NNP'), ('will', 'MD'), ('spot', 'VBN'), ('will', 'NNP')]

# list of untagged words
tagged_seq = Viterbi(test_tagged_words)

# Calculating accuracy
check = [i for i, j in zip(tagged_seq, expected_result) if i == j]
accuracy = len(check)/len(tagged_seq)

print(tagged_seq)
print("\nAccuracy : ", accuracy * 100)

[('Jack', 'MD'), ('will', 'NNP'), ('spot', 'VBN'), ('will', 'NNP')]

Accuracy :  50.0
